In [ ]:
%matplotlib inline
import os
import numpy as np
filepath='D:/measuring'
import scipy.signal
os.chdir(filepath)


In [ ]:
import analysis.scripts.cavity.oscilloscope_analysis_linewidth as oscillo_ana


In [ ]:

# Parameters for
g_a1 = 0.02
g_A1 = 0.02
g_x01 = 2.14
g_gamma1 = 0.1
g_A2 = 0.02
#g_x02 = 0.02
g_gamma2 = 0.1
g_dx = 0.24


